In [1]:
import os
import json
import numpy as np
import pandas as pd
from utils.eval import *
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt

with open("genre_symbols_mapping.json", "r") as file:
    genre_id_to_name = json.load(file)

with open("genreID_count_mapping.json", "r") as file:
    genre_Tcount = json.load(file)

In [2]:
a, b = 45, 140
tau = 0.13

accuracy_zero = evaluation_multi_segment("anchor_zero", "uni",a,b, tolerance= tau)
accuracy_peak= evaluation_multi_segment("anchor_peak", "uni",a,b, tolerance= tau)
accuracy_energy = evaluation_multi_segment("anchor_energy", "uni",a,b, tolerance= tau)


## Segment Set Ablation

In [ ]:

def octave_class(est, ref, tol=0.1):
    """
    Determine if estimate is half-time (-1), correct (0), or double-time (+1).
    tol = relative tolerance for BPM ratio, default ~5%.
    """
    ratio = est / ref
    if abs(ratio - 0.5) < tol:
        return -1
    elif abs(ratio - 1.0) < tol:
        return 0
    elif abs(ratio - 2.0) < tol:
        return 1
    else:
        return None  # off-grid

TOTAL_SAMPLES = 1341
def extract_stats(seg_dict):
    ref = np.array(seg_dict['hit_ref_bpm'])
    est = np.array(seg_dict['hit_bpm'])
    genres = np.array(seg_dict['hit_genres'])

    # accuracy
    acc1 = len(seg_dict['hit_idx']) / TOTAL_SAMPLES

    # octave
    octave = np.array([octave_class(e, r) for e, r in zip(est, ref)])

    # winning segment name
    winner_segments = seg_dict['hit_seg_names']   # list of strings

    return acc1, octave, genres, winner_segments

from collections import Counter, defaultdict
def summarize_segment_set(name, seg_dict, genre_id_to_name, genre_Tcount):
    acc1_global, octave, genres, winners = extract_stats(seg_dict)
    acc1_global *= 100

    # octave counts
    half = np.sum(octave == -1)
    corr = np.sum(octave == 0)
    dbl  = np.sum(octave == 1)

    # global winning-segment distribution
    global_win_counts = Counter(winners)

    # genre-wise accuracy + genre-wise winner distribution
    genrewise_acc = {}
    genrewise_winners = defaultdict(Counter)

    for wseg, gid in zip(winners, genres):
        gname = genre_id_to_name[str(gid)]
        genrewise_winners[gname][wseg] += 1

    for gid in np.unique(genres):
        gname = genre_id_to_name[str(gid)]
        correct_g = np.sum(genres == gid)
        total_g = genre_Tcount[gid]
        genrewise_acc[gname] = (correct_g / total_g) * 100

    return {
        "segment_set": name,
        "acc1": acc1_global,
        "half": half,
        "corr": corr,
        "double": dbl,
        "global_winners": dict(global_win_counts),
        "genrewise_acc": genrewise_acc,
        "genrewise_winners": {g: dict(c) for g, c in genrewise_winners.items()}
    }

results = []
segment_sets = {
    "F-T": accuracy_zero['bothfoot_y_torso_y_uni'],
    "H-F": accuracy_zero['bothhand_y_bothfoot_y_uni'],
    "H-T": accuracy_zero['bothhand_y_torso_y_uni'],
    "H-F-T": accuracy_zero['bothhand_y_bothfoot_y_torso_y_uni']
}


for name, seg_dict in segment_sets.items():
    r = summarize_segment_set(name, seg_dict, genre_id_to_name, genre_Tcount)
    results.append(r)



In [ ]:
for r in results:
    print(f"\n=== {r['segment_set']} ===")
    print(f"Global Acc1: {r['acc1']:.2f}%")
    print(f"Half: {r['half']} | Correct: {r['corr']} | Double: {r['double']}")
    
    print("\nGlobal Winning Segments:")
    for seg, count in r['global_winners'].items():
        print(f"  {seg:15s} {count}")

    print("\nGenre-wise Accuracy:")
    for g, acc in r['genrewise_acc'].items():
        print(f"  {g:15s} {acc:.1f}%")

    print("\nGenre-wise Winning Segments:")
    for g, wc in r['genrewise_winners'].items():
        print(f"  {g}: {wc}")



## FPS, Sparsity Ablation

In [ ]:
fps_15 = "./tempo_estimation_output_15fps/tempo_45_140/eval_data/multi/anchor_zero_uni.pkl"
fps_30 = "./tempo_estimation_output_30fps/tempo_45_140/eval_data/multi/anchor_zero_uni.pkl"

fps_15_data = load_pickle(fps_15)
fps_30_data = load_pickle(fps_30)

acc_fps_15 = fps_15_data['bothhand_y_bothfoot_y_torso_y_uni']['acc']
acc_fps_30 = fps_30_data['bothhand_y_bothfoot_y_torso_y_uni']['acc']


print(f"15 fps Acc1: {acc_fps_15:.2f}%")
print(f"30 fps Acc1: {acc_fps_30:.2f}%")



## Hop and Window

In [18]:
a, b = 45, 140
tau = 0.13

hop_list = [0.25, 0.5, 0.75, 1.0]   # percent
window_list = [1, 2, 3, 4, 5, 6, 7, 8]  # seconds
results = []

for hop in hop_list:
    for window in window_list:
    
        output_dir = f"./tempo_estimation_output_H{hop}_W{window}/tempo_{a}_{b}/"
        fname = os.path.join(output_dir, 'eval_data' , "multi", "anchor_zero_uni.pkl")
        
        if os.path.exists(fname):
            data = load_pickle(fname)
            acc = data['bothhand_y_bothfoot_y_torso_y_uni']['acc']
            # print(f"Acc1: {acc:.2f}%\n")

            # Save row
            results.append({
                "hop": hop,
                "window": window,
                "acc": round(acc,2)
            })
df = pd.DataFrame(results)
df.to_csv("./hop_window_accuracy.csv", index=False)